In [1]:
import torch  # This is all you need to use both PyTorch and TorchScript!
print(torch.__version__)

1.13.0+cu117


In [2]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()

    def forward(self, x, h):
        new_h = torch.tanh(x + h)
        return new_h, new_h

my_cell = MyCell()
x = torch.rand(3, 4)
h = torch.rand(3, 4)
print(my_cell(x, h))

(tensor([[0.6008, 0.5942, 0.5809, 0.8093],
        [0.9412, 0.8114, 0.1319, 0.5707],
        [0.6101, 0.7534, 0.7666, 0.7669]]), tensor([[0.6008, 0.5942, 0.5809, 0.8093],
        [0.9412, 0.8114, 0.1319, 0.5707],
        [0.6101, 0.7534, 0.7666, 0.7669]]))


In [3]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return new_h, new_h

my_cell = MyCell()
print(my_cell)
print(my_cell(x, h))

MyCell(
  (linear): Linear(in_features=4, out_features=4, bias=True)
)
(tensor([[ 0.7207, -0.1036,  0.5914, -0.1579],
        [ 0.8235,  0.2919,  0.2345, -0.4914],
        [ 0.5628,  0.3399,  0.7361, -0.6080]], grad_fn=<TanhBackward0>), tensor([[ 0.7207, -0.1036,  0.5914, -0.1579],
        [ 0.8235,  0.2919,  0.2345, -0.4914],
        [ 0.5628,  0.3399,  0.7361, -0.6080]], grad_fn=<TanhBackward0>))


In [4]:
class MyDecisionGate(torch.nn.Module):
    def forward(self, x):
        if x.sum() > 0:
            return x
        else:
            return -x

class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.dg = MyDecisionGate()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h, new_h

my_cell = MyCell()
print(my_cell)
print(my_cell(x, h))

MyCell(
  (dg): MyDecisionGate()
  (linear): Linear(in_features=4, out_features=4, bias=True)
)
(tensor([[ 0.6257,  0.1381,  0.4764,  0.7882],
        [ 0.7477,  0.4873, -0.1095,  0.6797],
        [ 0.2211,  0.6386,  0.5814,  0.5867]], grad_fn=<TanhBackward0>), tensor([[ 0.6257,  0.1381,  0.4764,  0.7882],
        [ 0.7477,  0.4873, -0.1095,  0.6797],
        [ 0.2211,  0.6386,  0.5814,  0.5867]], grad_fn=<TanhBackward0>))


In [5]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return new_h, new_h

my_cell = MyCell()
x, h = torch.rand(3, 4), torch.rand(3, 4)
traced_cell = torch.jit.trace(my_cell, (x, h))
print(traced_cell)
traced_cell(x, h)

MyCell(
  original_name=MyCell
  (linear): Linear(original_name=Linear)
)


(tensor([[ 0.6432,  0.8593,  0.5903,  0.2822],
         [ 0.8573,  0.7197,  0.0318, -0.4449],
         [ 0.7668,  0.8096, -0.0656, -0.6883]], grad_fn=<TanhBackward0>),
 tensor([[ 0.6432,  0.8593,  0.5903,  0.2822],
         [ 0.8573,  0.7197,  0.0318, -0.4449],
         [ 0.7668,  0.8096, -0.0656, -0.6883]], grad_fn=<TanhBackward0>))

In [6]:
print(traced_cell.graph)

graph(%self.1 : __torch__.MyCell,
      %x : Float(3, 4, strides=[4, 1], requires_grad=0, device=cpu),
      %h : Float(3, 4, strides=[4, 1], requires_grad=0, device=cpu)):
  %linear : __torch__.torch.nn.modules.linear.Linear = prim::GetAttr[name="linear"](%self.1)
  %20 : Tensor = prim::CallMethod[name="forward"](%linear, %x)
  %11 : int = prim::Constant[value=1]() # /tmp/ipykernel_65374/260609686.py:7:0
  %12 : Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu) = aten::add(%20, %h, %11) # /tmp/ipykernel_65374/260609686.py:7:0
  %13 : Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu) = aten::tanh(%12) # /tmp/ipykernel_65374/260609686.py:7:0
  %14 : (Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu), Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu)) = prim::TupleConstruct(%13, %13)
  return (%14)



In [7]:
print(traced_cell.code)

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  linear = self.linear
  _0 = torch.tanh(torch.add((linear).forward(x, ), h))
  return (_0, _0)



In [8]:
print(my_cell(x, h))
print(traced_cell(x, h))

(tensor([[ 0.6432,  0.8593,  0.5903,  0.2822],
        [ 0.8573,  0.7197,  0.0318, -0.4449],
        [ 0.7668,  0.8096, -0.0656, -0.6883]], grad_fn=<TanhBackward0>), tensor([[ 0.6432,  0.8593,  0.5903,  0.2822],
        [ 0.8573,  0.7197,  0.0318, -0.4449],
        [ 0.7668,  0.8096, -0.0656, -0.6883]], grad_fn=<TanhBackward0>))
(tensor([[ 0.6432,  0.8593,  0.5903,  0.2822],
        [ 0.8573,  0.7197,  0.0318, -0.4449],
        [ 0.7668,  0.8096, -0.0656, -0.6883]],
       grad_fn=<DifferentiableGraphBackward>), tensor([[ 0.6432,  0.8593,  0.5903,  0.2822],
        [ 0.8573,  0.7197,  0.0318, -0.4449],
        [ 0.7668,  0.8096, -0.0656, -0.6883]],
       grad_fn=<DifferentiableGraphBackward>))


In [9]:
class MyDecisionGate(torch.nn.Module):
    def forward(self, x):
        if x.sum() > 0:
            return x
        else:
            return -x

class MyCell(torch.nn.Module):
    def __init__(self, dg):
        super(MyCell, self).__init__()
        self.dg = dg
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h, new_h

my_cell = MyCell(MyDecisionGate())
traced_cell = torch.jit.trace(my_cell, (x, h))

print(traced_cell.dg.code)
print(traced_cell.code)

def forward(self,
    argument_1: Tensor) -> Tensor:
  return torch.neg(argument_1)

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  dg = self.dg
  linear = self.linear
  _0 = torch.add((dg).forward((linear).forward(x, ), ), h)
  _1 = torch.tanh(_0)
  return (_1, _1)



/tmp/ipykernel_65374/4234398751.py:3: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x.sum() > 0:


In [10]:
scripted_gate = torch.jit.script(MyDecisionGate())

my_cell = MyCell(scripted_gate)
scripted_cell = torch.jit.script(my_cell)

print(scripted_gate.code)
print(scripted_cell.code)

def forward(self,
    x: Tensor) -> Tensor:
  if bool(torch.gt(torch.sum(x), 0)):
    _0 = x
  else:
    _0 = torch.neg(x)
  return _0

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  dg = self.dg
  linear = self.linear
  _0 = torch.add((dg).forward((linear).forward(x, ), ), h)
  new_h = torch.tanh(_0)
  return (new_h, new_h)



In [11]:
# New inputs
x, h = torch.rand(3, 4), torch.rand(3, 4)
traced_cell(x, h)

(tensor([[0.8402, 0.3985, 0.5041, 0.7913],
         [0.6625, 0.6918, 0.2352, 0.6456],
         [0.6870, 0.2013, 0.8373, 0.2043]], grad_fn=<TanhBackward0>),
 tensor([[0.8402, 0.3985, 0.5041, 0.7913],
         [0.6625, 0.6918, 0.2352, 0.6456],
         [0.6870, 0.2013, 0.8373, 0.2043]], grad_fn=<TanhBackward0>))

In [12]:
class MyRNNLoop(torch.nn.Module):
    def __init__(self):
        super(MyRNNLoop, self).__init__()
        self.cell = torch.jit.trace(MyCell(scripted_gate), (x, h))

    def forward(self, xs):
        h, y = torch.zeros(3, 4), torch.zeros(3, 4)
        for i in range(xs.size(0)):
            y, h = self.cell(xs[i], h)
        return y, h

rnn_loop = torch.jit.script(MyRNNLoop())
print(rnn_loop.code)

def forward(self,
    xs: Tensor) -> Tuple[Tensor, Tensor]:
  h = torch.zeros([3, 4])
  y = torch.zeros([3, 4])
  y0 = y
  h0 = h
  for i in range(torch.size(xs, 0)):
    cell = self.cell
    _0 = (cell).forward(torch.select(xs, 0, i), h0, )
    y1, h1, = _0
    y0, h0 = y1, h1
  return (y0, h0)



In [13]:
class WrapRNN(torch.nn.Module):
    def __init__(self):
        super(WrapRNN, self).__init__()
        self.loop = torch.jit.script(MyRNNLoop())

    def forward(self, xs):
        y, h = self.loop(xs)
        return torch.relu(y)

traced = torch.jit.trace(WrapRNN(), (torch.rand(10, 3, 4)))
print(traced.code)

def forward(self,
    xs: Tensor) -> Tensor:
  loop = self.loop
  _0, y, = (loop).forward(xs, )
  return torch.relu(y)



In [14]:
traced.save('wrapped_rnn.pt')

loaded = torch.jit.load('wrapped_rnn.pt')

print(loaded)
print(loaded.code)

RecursiveScriptModule(
  original_name=WrapRNN
  (loop): RecursiveScriptModule(
    original_name=MyRNNLoop
    (cell): RecursiveScriptModule(
      original_name=MyCell
      (dg): RecursiveScriptModule(original_name=MyDecisionGate)
      (linear): RecursiveScriptModule(original_name=Linear)
    )
  )
)
def forward(self,
    xs: Tensor) -> Tensor:
  loop = self.loop
  _0, y, = (loop).forward(xs, )
  return torch.relu(y)

